In [1]:
import random

In [2]:
# Define a simple mock LLM class.
# This class simulates a Language Model that takes a prompt and returns a random predefined response.
# In a real-world scenario, this would be an actual LLM integration (e.g., OpenAI, HuggingFace).
class FakeLLM:
    def __init__(self):
        print('LLM created') # Prints a message when an instance of FakeLLM is created.

    # The 'predict' method simulates the LLM generating a response based on the input prompt.
    def predict(self, prompt):
        # A predefined list of possible responses.
        response_list = [
            'Paris is the capital of France',
            'NFL is a popular league',
            'AI stands for Artificial Intelligence'
        ]
        # Randomly selects one of the responses from the list.
        # In a real LLM, this would involve sending the prompt to the LLM API and getting its generated text.
        return {'response': random.choice(response_list)}

In [3]:
# Define a simple mock PromptTemplate class.
# This class simulates how a PromptTemplate works, taking a template string and input variables.
# It formats the template with provided values.
class FakePromptTemplate:
    def __init__(self, template, input_variables):
        self.template = template # The raw template string (e.g., 'Write a {length} poem about {topic}').
        self.input_variables = input_variables # A list of variable names expected in the template.

    # The 'format' method takes a dictionary of input values and formats the template string.
    def format(self, input_dict):
        # Uses string's format method to replace placeholders (e.g., {length}, {topic})
        # with values from the input_dict.
        return self.template.format(**input_dict)

In [4]:
# --- Example of using FakePromptTemplate and FakeLLM individually ---
# Create an instance of FakePromptTemplate with a specific template and variables.
template = FakePromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [5]:
# Format the template with actual values for 'length' and 'topic'.
prompt = template.format({'length':'short','topic':'USA'})
print(f"Formatted Prompt: {prompt}") # Output: Formatted Prompt: Write a short poem about USA

Formatted Prompt: Write a short poem about USA


In [6]:
# Create an instance of FakeLLM.
llm = FakeLLM() # Prints 'LLM created'

LLM created


In [7]:
# Make a prediction using the FakeLLM with the formatted prompt.
# This simulates calling an LLM API.
llm_response = llm.predict(prompt)
print(f"LLM Raw Response: {llm_response}") # Output: LLM Raw Response: {'response': '...' (one of the random choices)}

LLM Raw Response: {'response': 'Paris is the capital of France'}


In [8]:
# --- Introducing FakeLLMChain to combine PromptTemplate and LLM ---

# Define a simple mock LLMChain class.
# This class simulates a basic LangChain LLMChain, which orchestrates the flow
# from formatting a prompt to getting a prediction from an LLM.
class FakeLLMChain:
    def __init__(self, llm, prompt):
        self.llm = llm     # Stores the LLM instance (e.g., FakeLLM).
        self.prompt = prompt # Stores the PromptTemplate instance (e.g., FakePromptTemplate).

    # The 'run' method executes the chain's logic.
    # It takes a dictionary of input variables required by the prompt template.
    def run(self, input_dict):
        # 1. Format the prompt template using the provided input dictionary.
        final_prompt = self.prompt.format(input_dict)
        print(f"\nChain - Final Prompt Sent to LLM: {final_prompt}")

        # 2. Make a prediction using the LLM with the formatted prompt.
        result = self.llm.predict(final_prompt)
        print(f"Chain - LLM's Full Response: {result}")

        # 3. Extract and return the actual response text from the LLM's raw output.
        return result['response']

In [9]:
# --- Example of using FakeLLMChain ---
# IMPORTANT: The provided code had a typo 'NakliPromptTemplate'.
# We assume it's meant to be 'FakePromptTemplate' as defined above.
# If 'NakliPromptTemplate' was intended to be a separate class, it would need to be defined.
# For this explanation, we'll use FakePromptTemplate.
template_for_chain = FakePromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [10]:
llm_for_chain = FakeLLM() # Prints 'LLM created'
chain = FakeLLMChain(llm_for_chain, template_for_chain)

LLM created


In [11]:
# Run the chain with specific input variables.
# The chain handles formatting the prompt and getting the response from the LLM internally.
chain_result = chain.run({'length':'short', 'topic': 'india'})
print(f"Chain - Final Result: {chain_result}")


Chain - Final Prompt Sent to LLM: Write a short poem about india
Chain - LLM's Full Response: {'response': 'Paris is the capital of France'}
Chain - Final Result: Paris is the capital of France


## Understanding `FakeLLMChain` and its Parallel with LangChain's `LLMChain` 🤝

Dear Students,

The `FakeLLMChain` class you see above is a simplified, custom-made example that demonstrates the core concept behind **LangChain's `LLMChain`**. While it uses "fake" components, the fundamental workflow is identical to how a real `LLMChain` operates.

-----

### What is `FakeLLMChain` (and `LLMChain` in LangChain)?

At its heart, `FakeLLMChain` (and `LLMChain` in LangChain) serves as an **orchestrator** or a **pipeline** that connects a `PromptTemplate` to an `LLM` (Language Model). It automates the common two-step process of:

1.  **Formatting a prompt**: Taking user inputs and inserting them into a predefined prompt template.
2.  **Invoking the LLM**: Sending the fully formatted prompt to the Language Model to get a response.

-----

### Why We Use It (The Benefits) 🚀

Instead of performing these steps manually every time:

```python
# Manual approach
formatted_prompt = my_prompt_template.format(user_input)
llm_response = my_llm.invoke(formatted_prompt)
parsed_response = my_parser.parse(llm_response)
```

An `LLMChain` (or `FakeLLMChain`) encapsulates this logic, making your code:

  * **Cleaner and More Concise**: You define the chain once, and then simply call `run()` (or `invoke()` in modern LangChain) with your input.
  * **More Modular**: It clearly separates the concerns: the prompt template handles formatting, the LLM generates text, and the chain manages the flow between them.
  * **Easier to Reuse**: Once a chain is defined, you can reuse it anywhere in your application with different inputs without rewriting the prompt formatting and LLM calling logic.
  * **Foundation for Complex Workflows**: This basic chain is the building block for more advanced LangChain constructs like **Sequential Chains**, **Retrieval Chains**, and custom **LangChain Expression Language (LCEL)** pipelines.

-----

### How It Works (Step-by-Step) 👣

Let's break down the `FakeLLMChain`'s `run` method to see the parallel with `LLMChain`:

1.  **`final_prompt = self.prompt.format(input_dict)`**:

      * This line directly mirrors how `LLMChain` uses its internal `PromptTemplate`. When you call `chain.run({'length':'short', 'topic': 'india'})`, the `input_dict` is passed to the `FakePromptTemplate`'s `format` method.
      * The prompt template (`'Write a {length} poem about {topic}'`) gets filled in with `length='short'` and `topic='india'`, resulting in the actual prompt text sent to the LLM.

2.  **`result = self.llm.predict(final_prompt)`**:

      * This simulates the core interaction with the Language Model. In a real `LLMChain`, this would be where LangChain sends the `final_prompt` to the actual LLM (e.g., OpenAI's API, a HuggingFace model).
      * The `FakeLLM` here returns a predefined random response, but imagine a powerful LLM generating a unique poem or factual answer based on your prompt.

3.  **`return result['response']`**:

      * This step extracts the most relevant part of the LLM's output. Real LLM responses often come as objects or dictionaries with metadata. The chain extracts just the generated text content for you.

In essence, `FakeLLMChain` simplifies the interaction with `FakeLLM` and `FakePromptTemplate`, just as `LLMChain` (and more generally, LCEL chains) simplifies interactions with real LLMs and prompt templates in the LangChain ecosystem. It abstracts away the boilerplate, letting you focus on defining your prompts and what you want the LLM to achieve.